In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split

/Users/ernestsalim/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Preprocessing

In [2]:
train_ds, test_ds = tfds.load('emnist/letters', split = ['train', 'test'])

2023-07-17 10:25:56.781743: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2023-07-17 10:25:56.781787: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2023-07-17 10:25:56.781793: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2023-07-17 10:25:56.782389: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-07-17 10:25:56.782907: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
train_df = tfds.as_dataframe(train_ds)
test_df = tfds.as_dataframe(test_ds)

In [4]:
print(f'Training example size: {len(train_df)}')
print(f'Testing example size: {len(test_df)}')

Training example size: 88800
Testing example size: 14800


In [5]:
print(f'Shape of a single image data: {train_df.iloc[0, 0].shape}')

Shape of a single image data: (28, 28, 1)


In [32]:
train_df['image'][0].ravel().shape

(784,)

In [35]:
X_train = train_df['image'].apply(lambda a: a.flatten())
y_train = train_df['label']

X_test = test_df['image'].apply(lambda a: a.flatten())
y_test = test_df['label']

In [36]:
print(f'Shape of a flatten single image data: {X_train[0].shape}')

Shape of a flatten single image data: (784,)


In [54]:
X_train = np.vstack(X_train)
X_test = np.vstack(X_test)

In [55]:
print(f'Shape of input vector: {X_train.shape}')

Shape of input vector: (88800, 784)


In [60]:
letter_classes = y_train.unique()
letter_nums = len(letter_classes)

## Train Model

In [158]:
X_train = tf.keras.utils.normalize(X_train)
X_test = tf.keras.utils.normalize(X_test)

In [164]:
tf.random.set_seed(1234)
model = tf.keras.models.Sequential(layers = [
    tf.keras.Input(shape = (784, )),
    tf.keras.layers.Dense(units = 64, activation = 'relu'),
    tf.keras.layers.Dense(units = 128, activation = 'relu'),
    tf.keras.layers.Dense(units = letter_nums, activation = 'linear')
])

In [165]:
opt = tf.keras.optimizers.SGD(learning_rate = 0.002)
model.compile(optimizer = opt, loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True))

In [166]:
model.fit(X_train, y_train, epochs = 20)
model.save('Letter Handwriting Recognition')

Epoch 1/20
  38/2775 [..............................] - ETA: 7s - loss: 3.1348 

2023-07-17 12:23:59.118913: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-07-17 12:23:59.129049: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node SGD/AssignVariableOp.


2775/2775 [==============================] - 7s 3ms/step - loss: 3.1172
Epoch 2/20
2775/2775 [==============================] - 7s 3ms/step - loss: 3.0915
Epoch 3/20
2775/2775 [==============================] - 8s 3ms/step - loss: 3.0624
Epoch 4/20
2775/2775 [==============================] - 8s 3ms/step - loss: 3.0215
Epoch 5/20
2775/2775 [==============================] - 8s 3ms/step - loss: 2.9600
Epoch 6/20
2775/2775 [==============================] - 8s 3ms/step - loss: 2.8663
Epoch 7/20
2775/2775 [==============================] - 8s 3ms/step - loss: 2.7336
Epoch 8/20
2775/2775 [==============================] - 8s 3ms/step - loss: 2.5707
Epoch 9/20
2775/2775 [==============================] - 8s 3ms/step - loss: 2.3952
Epoch 10/20
2775/2775 [==============================] - 8s 3ms/step - loss: 2.2249
Epoch 11/20
2775/2775 [==============================] - 8s 3ms/step - loss: 2.0734
Epoch 12/20
2775/2775 [==============================] - 7s 3ms/step - loss: 1.9460
Epoch 13/20


INFO:tensorflow:Assets written to: Letter Handwriting Recognition/assets


## Load Saved Model

In [50]:
model = tf.keras.models.load_model('Digit Handwriting Recognition')

In [167]:
loss = model.evaluate(X_test, y_test)

  1/463 [..............................] - ETA: 1:55 - loss: 1.4418

2023-07-17 12:26:48.540718: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


463/463 [==============================] - 2s 4ms/step - loss: 1.6031


In [221]:
pred = model.predict(X_test)

463/463 [==============================] - 1s 1ms/step


In [222]:
y_hat = np.apply_along_axis(lambda arr: np.argmax(arr), 1, pred)

In [233]:
correct_predictions = np.sum(y_hat == y_test)

In [235]:
print(f'Test Accuracy: {correct_predictions / len(y_hat)}')

Test Accuracy: 0.5082432432432432


In [236]:
train_pred = model.predict(X_train)

2775/2775 [==============================] - 4s 1ms/step


In [237]:
y_train_hat = np.apply_along_axis(lambda arr: np.argmax(arr), 1, train_pred)

In [238]:
correct_predictions = np.sum(y_train_hat == y_train)

In [239]:
correct_predictions

44368

In [240]:
print(f'Test Accuracy: {correct_predictions / len(y_train)}')

Test Accuracy: 0.49963963963963964
